In [1]:
import os
import json
from pathlib import Path
import subprocess
project_root = Path.cwd().parent  

# نحدد المسارات الفرعية بناءً على Task-A
data_path = project_root / "data"
code_path = project_root / "code"
dev_set_file = Path(data_path) / "QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl"


In [2]:
def load_jsonl(input_path) -> list:
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data


def get_last_token_index(text):
    tokens = text.split()  # Split the string into tokens
    if tokens:
        last_token_index = len(tokens) - 1 # Get the last token index
        return last_token_index
    else:
        return -1  # Return -1 if the string has no tokens


def to_dict(answer, rank, score, start_token_indx, end_token_indx):
    return {
        # "pq_id": pq_id,
        "answer": answer,
        "strt_token_indx":start_token_indx,
        "end_token_indx":end_token_indx,
        "rank":rank,
        "score":score,
        }


def form_one_answer(passage):
    answers_list = []
    answers_list.append(to_dict(answer=passage, rank=1, score=1, start_token_indx=0, end_token_indx=get_last_token_index(passage)))
    return answers_list


### Whole passage baseline

In [6]:
dataset_jsonl  = load_jsonl(dev_set_file)

Loaded 163 records from C:\Users\sa\quran-qa-2023-main\Task-B\data\QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl


In [7]:
all_questions = {}
for pq_dict in dataset_jsonl:
    pq_id = pq_dict['pq_id']
    passage = pq_dict['passage']
    answers_list = form_one_answer(passage)
    all_questions.update({pq_id: answers_list})

save_path = Path(data_path) /"runs/whole_passage.json"
with open(save_path, "w", encoding="utf-8") as outfile:
    json.dump(all_questions, outfile, ensure_ascii=False) 
    print("Json file was saved into this path: ",save_path )


Json file was saved into this path:  C:\Users\sa\quran-qa-2023-main\Task-B\data\runs\whole_passage.json


### Evaluate the baseline

In [13]:

run_file = Path(data_path) / "runs/whole_passage.json"
gold_file = Path(data_path) / "QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl"

!python QQA23_TaskB_eval.py --run_file "{run_file}" --gold_answers_file "{gold_file}"

# expected output:
# Loaded 163 records from ../QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl
# pAP to all Results = 0.25484 


Loaded 163 records from C:\Users\sa\quran-qa-2023-main\Task-B\data\QQA23_TaskB_qrcd_v1.2_dev_preprocessed.jsonl
pAP@10 = 0.192
EM@1 = 0.006
F1@1 = 0.216
